## Apache Beam Portability Framework WordCount example

This notebook demonstrates how to run the Beam [wordcount.py](https://github.com/apache/beam/blob/v2.19.0/sdks/python/apache_beam/examples/wordcount.py) in Hopsworks.


## Do the imports and start the Beam jobservice

In [1]:
from __future__ import absolute_import

import argparse
import logging
import re

from past.builtins import unicode

import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.metrics import Metrics
from apache_beam.metrics.metric import MetricsFilter
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

import os
from hops import beam as hops_beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions, SetupOptions, HadoopFileSystemOptions, PortableOptions, WorkerOptions, DebugOptions

from hops import hdfs as hopsfs

# Start Beam jobservice
hops_beam.start(taskmanager_heap_size=8192)

{'jobserver_log': 'beamjobserver-flinking-runner-52504.log',
 'artifact_port': 14861,
 'expansion_port': 53886,
 'job_host': 'hopsworks0.logicalclocks.com',
 'job_port': 52504,
 'jobserver.pid': 24971}

## Get the portable runner pipeline options

In [2]:
pipeline_args = hops_beam.get_portable_runner_config()
pipeline_options=PipelineOptions(flags=pipeline_args)

## Run the WordCount example. 
Default **input** is the `README.md` file from the `Resources` dataset of this project and **output** is saved as `wordcount.out` in the same dataset.

In [3]:
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

class WordExtractingDoFn(beam.DoFn):
  """Parse each line of input text into words."""

  def __init__(self):
    # TODO(BEAM-6158): Revert the workaround once we can pickle super() on py3.
    # super(WordExtractingDoFn, self).__init__()
    beam.DoFn.__init__(self)
    self.words_counter = Metrics.counter(self.__class__, 'words')
    self.word_lengths_counter = Metrics.counter(self.__class__, 'word_lengths')
    self.word_lengths_dist = Metrics.distribution(
        self.__class__, 'word_len_dist')
    self.empty_line_counter = Metrics.counter(self.__class__, 'empty_lines')

  def process(self, element):
    """Returns an iterator over the words of this element.
    The element is a line of text.  If the line is blank, note that, too.
    Args:
      element: the element being processed
    Returns:
      The processed element.
    """
    text_line = element.strip()
    if not text_line:
      self.empty_line_counter.inc(1)
    words = re.findall(r'[\w\']+', text_line, re.UNICODE)
    for w in words:
      self.words_counter.inc()
      self.word_lengths_counter.inc(len(w))
      self.word_lengths_dist.update(len(w))
    return words

p = beam.Pipeline(options=pipeline_options)
 
  # Read the text file[pattern] into a PCollection.
lines = p | 'read' >> ReadFromText(os.path.join(hopsfs.project_path(exclude_nn_addr=True), 'Resources', 'README.md'))

  # Count the occurrences of each word.
def count_ones(word_ones):
    (word, ones) = word_ones
    return (word, sum(ones))

counts = (lines
        | 'split' >> (beam.ParDo(WordExtractingDoFn())
                      .with_output_types(unicode))
        | 'pair_with_one' >> beam.Map(lambda x: (x, 1))
        | 'group' >> beam.GroupByKey()
        | 'count' >> beam.Map(count_ones))

# Format the counts into a PCollection of strings.
def format_result(word_count):
    (word, count) = word_count
    return '%s: %d' % (word, count)

output = counts | 'format' >> beam.Map(format_result)

  # Write the output using a "Write" transform that has side effects.
  # pylint: disable=expression-not-assigned
output | 'write' >> WriteToText(os.path.join(hopsfs.project_path(exclude_nn_addr=True), 'Resources', 'wordcount.out'))

result = p.run()
result.wait_until_finish()

  # Do not query metrics when creating a template which doesn't run
if (not hasattr(result, 'has_job')    # direct runner
      or result.has_job):               # not just a template creation
    empty_lines_filter = MetricsFilter().with_name('empty_lines')
    query_result = result.metrics().query(empty_lines_filter)
    if query_result['counters']:
      empty_lines_counter = query_result['counters'][0]
      logging.info('number of empty lines: %d', empty_lines_counter.result)

    word_lengths_filter = MetricsFilter().with_name('word_len_dist')
    query_result = result.metrics().query(word_lengths_filter)
    if query_result['distributions']:
      word_lengths_dist = query_result['distributions'][0]
      logging.info('average word length: %d', word_lengths_dist.result.mean)
    
print('Done')

Done
